In [ ]:
import torch # for core funtionality , gives easy simple structure for NN , uses tensors as DATA STRUCTURES 
import torch.nn as nn # for neural networks
import torch.optim # for training netwrokssss
import math  # maths ops 
import copy # copying complex objects 

In [ ]:
class MultiHeadAttention(nn.Module): # nn.Module is base class for all models 
    def __init__( self, d_model, num_heads): 

        # d_model===> model's dimensions and num_heads===> refer to multiple sets of learned linear transformations applied to the input query, key, and value vectors, enabling the model to capture different aspects of the input sequence in parallel. Each head independently performs attention calculations, and the outputs are then combined, typically through concatenation followed by another linear transformation. 
        super(MultiHeadAttention, self).__init__()

        assert d_model % num_heads==0 # assert is used to verify a condition is true or not in the code and here we want d_model(model's dimensions) to be divisible by the num_heads

        # dimensions ko initialize karo

        self.d_model= d_model
        self.num_heads= num_heads
        self.d_k= d_model // num_heads # DIMENSION OF each head's key, query and value


        self.W_q= nn.Linear(d_model, d_model) # qyery transs
        self.W_k= nn.Linear(d_model, d_model) # key 
        self.W_v= nn.Linear(d_model, d_model) # value
        self.W_o= nn.Linear(d_model, d_model) # last mai output


In [ ]:
# Scaled dot product attention which basically finds relation between input words

def scaled_dot_product_attention(self, Q, K, V, mask=None):

    attn_scores= torch.matmul(Q, K.transpose(-2,-1))/ math.sqrt(self.self.d_k) # matmul for Matrix product of two tensors.

    if mask is not None:      # Apply mask if provided (useful for preventing attention to certain parts like padding)

        attn_scores= attn_scores.masked_fill(mask==0, -1e9)

    attn_probs= torch.softmax(attn_scores, dim=1)     # softmax for attention ko probabilities mai convert karne kelie that sums to 1 

    output = torch.matmul(attn_probs, V)       # matmul final O/P
    return output    



def split_heads(self, x): #split kiya heads ko 
    batch_size, seq_length, d_model= x.size()
    return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1,2)


def combine_heads(self,x): # yahan concat kiya multiple heads ka result

    batch_size, _, seq_length, d_k= x.size()
    return x.transpose(1,2).contiguous().view(batch_size, seq_length, self.d_model)

def forward(self, Q, K, V, mask=None): # method applies linear transformations, splits the heads, performs scaled dot-product attention, combines the heads, and returns the output.
    Q= self.split_heads(self.W_q(Q)) # splitting and multiplying with learnable weights
    K= self.split_heads(self.W_k(K))
    V= self.split_heads(self.W_v(V))

    attn_output=self.scaled_dot_product_attention( Q, K, V, mask) # DOT PRODUCT CALL KIYA 

    output= self.W_o(self.combine_heads(attn_output))
    return output



In summary, the MultiHeadAttention class encapsulates the multi-head attention mechanism commonly used in transformer models. It takes care of splitting the input into multiple attention heads, applying attention to each head, and then combining the results. By doing so, the model can capture various relationships in the input data at different scales, improving the expressive ability of the model.